## Process agg data


In [1]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

from datetime import datetime, timedelta
import time 
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
import pyarrow.parquet as pq
import pyarrow as pa

In [3]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [5]:
train_file = r"amex\agg_train_all.parquet"

In [6]:
%%time
df=pd.read_parquet(train_file, engine='pyarrow')


Wall time: 3.84 s


In [7]:
df.head(2)

,customer_ID,D_104,D_73,D_77,R_24,D_145,D_123,B_31=0,B_31=1,D_91,B_2,D_135,R_10,S_20,R_21,D_112,D_124,D_63=CL,D_63=CO,D_63=CR,D_63=XL,D_63=XM,D_63=XZ,S_11,D_125,D_46,R_1,R_6,B_3,R_12,D_106,B_18,S_15,S_12,B_6,B_22,D_41,D_103,D_108,B_39,S_16,D_120=0.0,D_120=1.0,S_23,S_2=min,S_2=max,days,S_18,R_2,D_131,R_25,B_5,B_25,D_52,D_53,D_89,D_144,D_78,P_2,D_56,D_119,P_3,B_16,S_19,B_15,D_137,D_122,D_141,D_142,D_43,B_9,D_45,S_24,D_58,S_7,R_22,D_143,B_30=0.0,B_30=1.0,B_30=2.0,B_27,D_66=0.0,D_66=1.0,B_36,D_139,D_118,D_114=0.0,D_114=1.0,B_4,B_24,D_102,S_6,D_136,B_33,D_138,D_105,R_3,B_11,D_47,D_59,B_1,D_60,S_5,B_41,B_14,D_50,R_7,B_8,D_134,D_132,D_127,R_4,D_65,B_23,B_42,D_74,D_79,D_130,B_19,D_68=0.0,D_68=1.0,D_68=2.0,D_68=3.0,D_68=4.0,D_68=5.0,D_68=6.0,S_8,R_17,P_4,D_81,D_128,B_37,R_15,D_70,D_71,B_32,S_9,D_93,R_8,S_22,D_61,B_7,D_86,B_26,B_20,S_13,D_126=-1.0,D_126=0.0,D_126=1.0,D_69,R_16,0,D_109,D_64=-1,D_64=O,D_64=R,D_64=U,D_113,R_23,R_13,B_29,D_133,D_49,B_17,D_39,D_87,D_117=-1.0,D_117=1.0,D_117=2.0,D_117=3.0,D_117=4.0,D_117=5.0,D_117=6.0,D_140,S_3,D_62,D_72,R_19,D_80,D_54,B_38=1.0,B_38=2.0,B_38=3.0,B_38=4.0,B_38=5.0,B_38=6.0,B_38=7.0,D_84,D_116=0.0,D_116=1.0,D_75,D_83,D_111,D_121,D_110,R_26,R_20,D_82,B_13,R_18,D_88,S_26,B_40,R_14,B_21,D_48,R_11,S_17,R_28,R_9,D_92,S_27,D_76,D_96,D_115,D_42,R_5,B_10,D_94,R_27,B_12,D_55,D_107,D_44,S_25,D_51,B_28,D_129
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.009606,NaN,0.419295,0.005469,0.005814,0.003103,0.0,13.0,1.197627,1.005086,NaN,0.004742,0.006292,0.004931,1.004348,0.686641,0.0,0.0,13.0,0.0,0.0,0.0,0.469887,0.006253,0.378074,0.004509,0.006067,0.006456,1.004889,NaN,0.842565,0.173335,0.247988,0.11351,0.005700,0.005021,1.005188,NaN,NaN,0.004974,13.0,0.0,0.135036,2017-03-09,2018-03-13,369,0.005075,0.005754,0.005446,0.005799,0.146650,0.005915,0.204972,NaN,0.005235,0.005283,0.005218,0.933824,0.158571,0.244733,0.680138,0.004769,0.005955,0.026247,NaN,0.433732,0.005178,NaN,NaN,0.006220,0.725369,0.931956,0.064803,0.098374,0.005002,0.005066,13.0,0.0,0.0,0.004098,NaN,NaN,0.005292,0.003664,0.245514,0.0,13.0,0.040309,0.005135,0.856909,0.004647,NaN,1.004033,NaN,0.984966,0.004752,0.007230,0.532874,0.061147,0.012007,0.534817,0.029112,0.006753,0.023142,0.150326,0.005851,0.003585,NaN,NaN,1.006123,0.006656,0.004701,0.026177,NaN,0.039496,0.004170,0.005195,0.005231,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.797145,0.004164,0.005670,0.005834,1.004154,0.012015,0.005171,0.005221,0.214785,0.005084,0.039818,0.004415,0.005595,0.916144,0.225847,0.036624,0.004874,0.004408,0.005731,0.654700,0.0,0.0,13.0,0.005275,0.004740,13,0.006114,0.0,13.0,0.0,0.0,0.005476,0.004593,0.004522,NaN,0.004760,NaN,NaN,0.010704,NaN,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.005343,0.113215,0.181286,0.004495,0.005231,0.081974,1.006225,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.004945,13.0,0.0,0.036125,0.004892,NaN,0.711829,NaN,NaN,0.005231,0.505637,0.100432,0.006589,NaN,0.012560,0.087300,0.005879,0.005500,0.240978,0.004784,0.005079,0.004089,NaN,0.543890,0.850951,NaN,0.003619,0.247095,NaN,0.004684,0.270280,0.005909,1.005594,0.125683,0.224432,0.672166,0.004673,0.974669,0.978914,0.050968,1.005537
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.004932,NaN,0.223911,0.004027,0.004902,0.006022,0.0,13.0,0.082667,0.991083,NaN,0.005269,0.005367,0.004984,1.006037,0.140331,0.0,13.0,0.0,0.0,0.0,0.0,0.375552,0.004236,0.452041,0.006246,0.005476,0.005663,1.005214,NaN,1.004884,0.265484,0.181548,0.20227,0.005374,0.004993,0.005606,NaN,NaN,0.004212,8.0,5.0,0.135614,2017-03-01,2018-03-25,389,0.006049,0.006251,0.004214,0.004724,0.035462,0.024090,0.158313,NaN,0.003807,0.004218,0.003896,0.899820,0.705671,0.430961,0.566665,0.076456,0.005197,0.005560,NaN,0.290804,0.006007,NaN,0.144571,0.010298,0.256461,0.910228,0.005146,0.103002,0.003541,0.004824,13.0,0.0,0.0,0.004588,NaN,NaN,0.006088,0.004906,0.433039,0.0,13.0,0.016746,0.004736,0.004578,0.003821,NaN,1.005387,NaN,NaN,0.057673,0.013792,0.392433,0.232975,0.025654,0.326530,0.016785,0.005432,0.014848,NaN,0.005768,0.004956,NaN,NaN,0.080648,0.003229,0.004130,0.0

In [8]:
df.rename(columns={'0':'cnt'}, inplace=True)

In [9]:
all_cols = ['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_31=0', 'B_31=1', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38=1.0', 'B_38=2.0', 'B_38=3.0', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_114=0.0', 'D_114=1.0', 'D_115', 'D_116=0.0', 'D_116=1.0', 'D_117=-1.0', 'D_117=1.0', 'D_117=2.0', 'D_117=3.0', 'D_117=4.0', 'D_117=5.0', 'D_117=6.0', 'D_118', 'D_119', 'D_120=0.0', 'D_120=1.0', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126=-1.0', 'D_126=0.0', 'D_126=1.0', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63=CL', 'D_63=CO', 'D_63=CR', 'D_63=XL', 'D_63=XM', 'D_63=XZ', 'D_64=-1', 'D_64=O', 'D_64=R', 'D_64=U', 'D_65', 'D_66=0.0', 'D_66=1.0', 'D_68=0.0', 'D_68=1.0', 'D_68=2.0', 'D_68=3.0', 'D_68=4.0', 'D_68=5.0', 'D_68=6.0', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96', 'P_2', 'P_3', 'P_4', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 'R_21', 'R_22', 'R_23', 'R_24', 'R_25', 'R_26', 'R_27', 'R_28', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_2=max', 'S_2=min', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'customer_ID', 'days']

cat_feats = ['B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_31=0', 'B_31=1', 'B_38=1.0', 'B_38=2.0', 'B_38=3.0', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'D_114=0.0', 'D_114=1.0', 'D_116=0.0', 'D_116=1.0', 'D_117=-1.0', 'D_117=1.0', 'D_117=2.0', 'D_117=3.0', 'D_117=4.0', 'D_117=5.0', 'D_117=6.0', 'D_120=0.0', 'D_120=1.0', 'D_126=-1.0', 'D_126=0.0', 'D_126=1.0', 'D_63=CL', 'D_63=CO', 'D_63=CR', 'D_63=XL', 'D_63=XM', 'D_63=XZ', 'D_64=-1', 'D_64=O', 'D_64=R', 'D_64=U', 'D_66=0.0', 'D_66=1.0', 'D_68=0.0', 'D_68=1.0', 'D_68=2.0', 'D_68=3.0', 'D_68=4.0', 'D_68=5.0', 'D_68=6.0']
s2_feats = ['S_2=max', 'S_2=min']
float_feats = ['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_32', 'B_33', 'B_36', 'B_37', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96', 'P_2', 'P_3', 'P_4', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 'R_21', 'R_22', 'R_23', 'R_24', 'R_25', 'R_26', 'R_27', 'R_28', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'days']

to_log_feats = ['B_11', 'B_12', 'B_13', 'B_21', 'B_22','B_23', 'B_24', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_32', 'B_36', 
             'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_9', 
             'D_106', 'D_107', 'D_108', 'D_109', 'D_113', 'D_115', 'D_118', 'D_119', 'D_123', 'D_125', 'D_131', 'D_133', 
             'D_135', 'D_136', 'D_137', 'D_138', 'D_140', 
             'D_39', 'D_41', 'D_43', 'D_44',  'D_45', 'D_49','D_51']
print(float_feats)
print(to_log_feats)

['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_32', 'B_33', 'B_36', 'B_37', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D

In [10]:
eps =  1e-8

log_feats = []
for c in to_log_feats:
    df[f'log_{c}'] = np.log(df[c].values + eps)
    log_feats.append(f'log_{c}')

In [11]:
print(log_feats)

['log_B_11', 'log_B_12', 'log_B_13', 'log_B_21', 'log_B_22', 'log_B_23', 'log_B_24', 'log_B_26', 'log_B_27', 'log_B_28', 'log_B_29', 'log_B_3', 'log_B_32', 'log_B_36', 'log_B_4', 'log_B_40', 'log_B_41', 'log_B_42', 'log_B_5', 'log_B_9', 'log_D_106', 'log_D_107', 'log_D_108', 'log_D_109', 'log_D_113', 'log_D_115', 'log_D_118', 'log_D_119', 'log_D_123', 'log_D_125', 'log_D_131', 'log_D_133', 'log_D_135', 'log_D_136', 'log_D_137', 'log_D_138', 'log_D_140', 'log_D_39', 'log_D_41', 'log_D_43', 'log_D_44', 'log_D_45', 'log_D_49', 'log_D_51']


In [12]:
df.shape

(458913, 272)

In [13]:
df.head(2)

,customer_ID,D_104,D_73,D_77,R_24,D_145,D_123,B_31=0,B_31=1,D_91,B_2,D_135,R_10,S_20,R_21,D_112,D_124,D_63=CL,D_63=CO,D_63=CR,D_63=XL,D_63=XM,D_63=XZ,S_11,D_125,D_46,R_1,R_6,B_3,R_12,D_106,B_18,S_15,S_12,B_6,B_22,D_41,D_103,D_108,B_39,S_16,D_120=0.0,D_120=1.0,S_23,S_2=min,S_2=max,days,S_18,R_2,D_131,R_25,B_5,B_25,D_52,D_53,D_89,D_144,D_78,P_2,D_56,D_119,P_3,B_16,S_19,B_15,D_137,D_122,D_141,D_142,D_43,B_9,D_45,S_24,D_58,S_7,R_22,D_143,B_30=0.0,B_30=1.0,B_30=2.0,B_27,D_66=0.0,D_66=1.0,B_36,D_139,D_118,D_114=0.0,D_114=1.0,B_4,B_24,D_102,S_6,D_136,B_33,D_138,D_105,R_3,B_11,D_47,D_59,B_1,D_60,S_5,B_41,B_14,D_50,R_7,B_8,D_134,D_132,D_127,R_4,D_65,B_23,B_42,D_74,D_79,D_130,B_19,D_68=0.0,D_68=1.0,D_68=2.0,D_68=3.0,D_68=4.0,D_68=5.0,D_68=6.0,S_8,R_17,P_4,D_81,D_128,B_37,R_15,D_70,D_71,B_32,S_9,D_93,R_8,S_22,D_61,B_7,D_86,B_26,B_20,S_13,D_126=-1.0,D_126=0.0,D_126=1.0,D_69,R_16,cnt,D_109,D_64=-1,D_64=O,D_64=R,D_64=U,D_113,R_23,R_13,B_29,D_133,D_49,B_17,D_39,D_87,D_117=-1.0,D_117=1.0,D_117=2.0,D_117=3.0,D_117=4.0,D_117=5.0,D_117=6.0,D_140,S_3,D_62,D_72,R_19,D_80,D_54,B_38=1.0,B_38=2.0,B_38=3.0,B_38=4.0,B_38=5.0,B_38=6.0,B_38=7.0,D_84,D_116=0.0,D_116=1.0,D_75,D_83,D_111,D_121,D_110,R_26,R_20,D_82,B_13,R_18,D_88,S_26,B_40,R_14,B_21,D_48,R_11,S_17,R_28,R_9,D_92,S_27,D_76,D_96,D_115,D_42,R_5,B_10,D_94,R_27,B_12,D_55,D_107,D_44,S_25,D_51,B_28,D_129,log_B_11,log_B_12,log_B_13,log_B_21,log_B_22,log_B_23,log_B_24,log_B_26,log_B_27,log_B_28,log_B_29,log_B_3,log_B_32,log_B_36,log_B_4,log_B_40,log_B_41,log_B_42,log_B_5,log_B_9,log_D_106,log_D_107,log_D_108,log_D_109,log_D_113,log_D_115,log_D_118,log_D_119,log_D_123,log_D_125,log_D_131,log_D_133,log_D_135,log_D_136,log_D_137,log_D_138,log_D_140,log_D_39,log_D_41,log_D_43,log_D_44,log_D_45,log_D_49,log_D_51
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.009606,NaN,0.419295,0.005469,0.005814,0.003103,0.0,13.0,1.197627,1.005086,NaN,0.004742,0.006292,0.004931,1.004348,0.686641,0.0,0.0,13.0,0.0,0.0,0.0,0.469887,0.006253,0.378074,0.004509,0.006067,0.006456,1.004889,NaN,0.842565,0.173335,0.247988,0.11351,0.005700,0.005021,1.005188,NaN,NaN,0.004974,13.0,0.0,0.135036,2017-03-09,2018-03-13,369,0.005075,0.005754,0.005446,0.005799,0.146650,0.005915,0.204972,NaN,0.005235,0.005283,0.005218,0.933824,0.158571,0.244733,0.680138,0.004769,0.005955,0.026247,NaN,0.433732,0.005178,NaN,NaN,0.006220,0.725369,0.931956,0.064803,0.098374,0.005002,0.005066,13.0,0.0,0.0,0.004098,NaN,NaN,0.005292,0.003664,0.245514,0.0,13.0,0.040309,0.005135,0.856909,0.004647,NaN,1.004033,NaN,0.984966,0.004752,0.007230,0.532874,0.061147,0.012007,0.534817,0.029112,0.006753,0.023142,0.150326,0.005851,0.003585,NaN,NaN,1.006123,0.006656,0.004701,0.026177,NaN,0.039496,0.004170,0.005195,0.005231,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.797145,0.004164,0.005670,0.005834,1.004154,0.012015,0.005171,0.005221,0.214785,0.005084,0.039818,0.004415,0.005595,0.916144,0.225847,0.036624,0.004874,0.004408,0.005731,0.654700,0.0,0.0,13.0,0.005275,0.004740,13,0.006114,0.0,13.0,0.0,0.0,0.005476,0.004593,0.004522,NaN,0.004760,NaN,NaN,0.010704,NaN,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.005343,0.113215,0.181286,0.004495,0.005231,0.081974,1.006225,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.004945,13.0,0.0,0.036125,0.004892,NaN,0.711829,NaN,NaN,0.005231,0.505637,0.100432,0.006589,NaN,0.012560,0.087300,0.005879,0.005500,0.240978,0.004784,0.005079,0.004089,NaN,0.543890,0.850951,NaN,0.003619,0.247095,NaN,0.004684,0.270280,0.005909,1.005594,0.125683,0.224432,0.672166,0.004673,0.974669,0.978914,0.050968,1.005537,-4.929449,-2.073995,-2.298275,-5.203047,-5.167329,-3.642872,-5.271716,-5.424415,-5.497164,-2.976560,NaN,-5.042712,-5.281577,-5.241639,-3.211186,-2.438409,-4.997762,NaN,-1.919705,-5.079943,NaN,-0.397250,NaN,-5.097225,-5.207382,-1.397983,-1.404402,-1.407586,-5.775453,-5.074720,-5.212901,-5.347446,NaN,NaN,NaN,NaN,-5.232025,-4.537166,-5.294216,NaN,-5.365879,-0.321075,NaN,-0.021312
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.004932,NaN,0.223911,0.004027,0.004902,0.006022,0.0,13.0,0.082667,

In [14]:
train_stats_file = r"amex\train_agg_stats.csv"
agg_stats = pd.read_csv(train_stats_file, sep='|', index_col=0)
agg_stats.head(2)

,feat,min,max,mean,std,median,skew,kurtosis,quantile0.05,quantile0.15,quantile0.25,quantile0.75,quantile0.85,quantile0.95
0,D_144,0.000004,1.342895,0.051679,0.173223,0.005127,4.289567,18.931978,0.003598,0.004168,0.004498,0.005852,0.006462,0.415345
1,S_26,0.000002,84.508010,0.063837,0.411714,0.005889,75.542442,10092.106445,0.003786,0.004434,0.004844,0.032713,0.078652,0.253601


In [15]:
## fill na
for c in cat_feats:
    df[c] = df[c].fillna(0)


for _, row in agg_stats.iterrows():
    c = row['feat']
    avg = row['mean']

    if c in float_feats + log_feats:
        df[c] = df[c].fillna(value=avg)
        
for c in log_feats:
    df[c] = df[c].fillna(value=df[c].mean())

In [16]:
float64_cols = df.select_dtypes(include=['float64']).columns.tolist()
df[float64_cols] = np.float32(df[float64_cols].values)
#---convert int64 to int32
int64_cols = df.select_dtypes(include=['int64']).columns.tolist()
df[int64_cols] = np.int32(df[int64_cols].values)

In [17]:
%%time
train_labels = pd.read_csv(r'amex\train_labels.csv')
print(train_labels.shape)
display(train_labels.head(2))

(458913, 2)


,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0


Wall time: 560 ms


In [18]:
df = df.merge(train_labels, on=['customer_ID'], how='inner')
df.shape

(458913, 273)

In [19]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 273 entries, customer_ID to target
dtypes: float32(267), int32(2), int64(1), object(3)
memory usage: 488.4+ MB


In [21]:
pq.write_table(pa.Table.from_pandas(df), r'amex\agg_train_all_rev.parquet', compression = 'GZIP')

In [22]:
x_cols = float_feats + cat_feats + log_feats
print(len(x_cols))
print(x_cols)

269
['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_32', 'B_33', 'B_36', 'B_37', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91'

## Process test agg data


In [23]:
test_file = r"amex\agg_test_all.parquet"

In [24]:
%%time
df=pd.read_parquet(test_file, engine='pyarrow')


Wall time: 8.14 s


In [25]:
df.head(2)

,customer_ID,D_104,D_73,D_77,R_24,D_145,D_123,B_31=0,B_31=1,D_91,B_2,D_135,R_10,S_20,R_21,D_112,D_124,D_63=CL,D_63=CO,D_63=CR,D_63=XL,D_63=XM,D_63=XZ,S_11,D_125,D_46,R_1,R_6,B_3,R_12,D_106,B_18,S_15,S_12,B_6,B_22,D_41,D_103,D_108,B_39,S_16,D_120=0.0,D_120=1.0,S_23,S_2=min,S_2=max,days,S_18,R_2,D_131,R_25,B_5,B_25,D_52,D_53,D_89,D_144,D_78,P_2,D_56,D_119,P_3,B_16,S_19,B_15,D_137,D_122,D_141,D_142,D_43,B_9,D_45,S_24,D_58,S_7,R_22,D_143,B_30=0.0,B_30=1.0,B_30=2.0,B_27,D_66=1.0,B_36,D_139,D_118,D_114=0.0,D_114=1.0,B_4,B_24,D_102,S_6,D_136,B_33,D_138,D_105,R_3,B_11,D_47,D_59,B_1,D_60,S_5,B_41,B_14,D_50,R_7,B_8,D_134,D_132,D_127,R_4,D_65,B_23,B_42,D_74,D_79,D_130,B_19,D_68=1.0,D_68=2.0,D_68=3.0,D_68=4.0,D_68=5.0,D_68=6.0,S_8,R_17,P_4,D_81,D_128,B_37,R_15,D_70,D_71,B_32,S_9,D_93,R_8,S_22,D_61,B_7,D_86,B_26,B_20,S_13,D_126=-1.0,D_126=0.0,D_126=1.0,D_69,R_16,0,D_109,D_64=O,D_64=R,D_64=U,D_113,R_23,R_13,B_29,D_133,D_49,B_17,D_39,D_87,D_117=-1.0,D_117=1.0,D_117=2.0,D_117=3.0,D_117=4.0,D_117=5.0,D_117=6.0,D_140,S_3,D_62,D_72,R_19,D_80,D_54,B_38=1.0,B_38=2.0,B_38=3.0,B_38=4.0,B_38=5.0,B_38=6.0,B_38=7.0,D_84,D_116=0.0,D_116=1.0,D_75,D_83,D_111,D_121,D_110,R_26,R_20,D_82,B_13,R_18,D_88,S_26,B_40,R_14,B_21,D_48,R_11,S_17,R_28,R_9,D_92,S_27,D_76,D_96,D_115,D_42,R_5,B_10,D_94,R_27,B_12,D_55,D_107,D_44,S_25,D_51,B_28,D_129
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.005923,0.182713,0.002916,0.006416,0.005271,0.003728,0.0,9.0,0.002312,0.898289,NaN,0.005161,0.005406,0.003851,1.004091,0.008423,0.0,0.0,9.0,0.0,0.0,0.0,1.608184,0.005669,0.446102,0.005773,0.003876,0.009732,1.004421,NaN,0.590665,0.459957,0.136646,0.218059,0.004888,0.002635,0.004177,NaN,NaN,0.005912,0.0,5.0,0.136015,2019-02-19,2019-10-12,235,0.00601,0.005000,0.004620,0.005308,0.047263,0.004567,0.117639,NaN,0.004934,0.005730,0.004126,0.601387,NaN,0.247668,0.591933,0.003792,0.006266,0.020536,NaN,0.145303,0.005657,NaN,0.006835,0.003733,0.011955,0.949735,0.377217,0.170330,0.004632,0.005480,9.0,0.0,0.0,0.005720,NaN,0.004009,0.004483,0.246912,5.0,0.0,0.132557,0.005203,0.005494,0.227071,NaN,1.005949,NaN,NaN,0.104659,0.006187,0.479550,0.412407,0.013925,0.042616,0.006742,0.006821,0.018828,NaN,0.004423,1.006641,NaN,NaN,0.005166,0.005101,0.004591,0.254266,NaN,0.219199,0.004534,0.005501,0.004468,0.0,0.0,0.0,0.0,0.0,5.0,0.271142,0.004437,0.961646,0.004031,0.006024,0.013483,0.003600,0.004415,0.072740,0.005471,0.01557,0.004286,0.006230,0.962658,0.597552,0.283370,0.007949,0.005206,0.005869,0.601419,0.0,9.0,0.0,0.006233,0.004303,9,0.004123,0.0,0.0,5.0,0.005696,0.005964,0.004435,0.003894,0.004254,NaN,NaN,0.069758,NaN,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003996,0.188640,0.035462,0.004783,0.003948,0.005002,1.005194,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.004711,5.0,0.0,0.204130,0.005146,NaN,0.196751,NaN,NaN,0.004758,0.504131,0.054207,0.003736,NaN,0.038056,0.368253,0.004547,0.007769,0.578279,0.005572,0.005725,0.005292,NaN,0.116208,0.299671,NaN,0.004607,0.252104,0.118016,0.004464,0.037069,0.005423,1.006624,0.082867,0.317071,0.002802,0.007083,0.973917,0.041620,0.14522,0.006681
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.005631,NaN,0.304190,0.005082,0.005466,0.003205,0.0,13.0,0.235882,1.003704,NaN,0.004744,0.005877,0.004694,1.005064,0.641199,0.0,13.0,0.0,0.0,0.0,0.0,0.173136,0.004250,0.454435,0.006147,0.003863,0.006160,1.005026,NaN,1.005336,0.106175,0.214804,0.195901,0.004317,0.003970,0.003439,NaN,NaN,0.005592,13.0,0.0,0.135646,2018-04-22,2019-04-15,358,0.00477,0.005472,0.004196,0.004663,0.101462,0.058109,0.148449,NaN,0.004422,0.005329,0.004939,0.862166,0.064233,0.303837,0.553269,0.051158,0.005619,0.004710,NaN,0.576952,0.004814,NaN,NaN,0.081637,0.088032,0.920595,0.005272,0.078633,0.004727,0.005762,13.0,0.0,0.0,0.005269,NaN,0.005196,0.006455,0.303904,0.0,13.0,0.061213,0.005254,0.004841,0.004829,NaN,1.003672,NaN,NaN,0.005181,0.035716,0.351355,0.583729,0.052342,0.507825,0.022617,0.005565,0.034238,NaN,0.005995,1.004206,NaN,NaN,0.006148,0.005043,0.004693,0.022771,NaN,0.004456,0.005173,0.005076,0.004885,0.0,0.0,0

In [26]:
df.rename(columns={'0':'cnt'}, inplace=True)

In [27]:
eps =  1e-8

log_feats = []
for c in to_log_feats:
    df[f'log_{c}'] = np.log(df[c].values + eps)
    log_feats.append(f'log_{c}')

In [28]:
print(log_feats)

['log_B_11', 'log_B_12', 'log_B_13', 'log_B_21', 'log_B_22', 'log_B_23', 'log_B_24', 'log_B_26', 'log_B_27', 'log_B_28', 'log_B_29', 'log_B_3', 'log_B_32', 'log_B_36', 'log_B_4', 'log_B_40', 'log_B_41', 'log_B_42', 'log_B_5', 'log_B_9', 'log_D_106', 'log_D_107', 'log_D_108', 'log_D_109', 'log_D_113', 'log_D_115', 'log_D_118', 'log_D_119', 'log_D_123', 'log_D_125', 'log_D_131', 'log_D_133', 'log_D_135', 'log_D_136', 'log_D_137', 'log_D_138', 'log_D_140', 'log_D_39', 'log_D_41', 'log_D_43', 'log_D_44', 'log_D_45', 'log_D_49', 'log_D_51']


In [30]:
df.shape

(924621, 269)

In [31]:
df.head(2)

,customer_ID,D_104,D_73,D_77,R_24,D_145,D_123,B_31=0,B_31=1,D_91,B_2,D_135,R_10,S_20,R_21,D_112,D_124,D_63=CL,D_63=CO,D_63=CR,D_63=XL,D_63=XM,D_63=XZ,S_11,D_125,D_46,R_1,R_6,B_3,R_12,D_106,B_18,S_15,S_12,B_6,B_22,D_41,D_103,D_108,B_39,S_16,D_120=0.0,D_120=1.0,S_23,S_2=min,S_2=max,days,S_18,R_2,D_131,R_25,B_5,B_25,D_52,D_53,D_89,D_144,D_78,P_2,D_56,D_119,P_3,B_16,S_19,B_15,D_137,D_122,D_141,D_142,D_43,B_9,D_45,S_24,D_58,S_7,R_22,D_143,B_30=0.0,B_30=1.0,B_30=2.0,B_27,D_66=1.0,B_36,D_139,D_118,D_114=0.0,D_114=1.0,B_4,B_24,D_102,S_6,D_136,B_33,D_138,D_105,R_3,B_11,D_47,D_59,B_1,D_60,S_5,B_41,B_14,D_50,R_7,B_8,D_134,D_132,D_127,R_4,D_65,B_23,B_42,D_74,D_79,D_130,B_19,D_68=1.0,D_68=2.0,D_68=3.0,D_68=4.0,D_68=5.0,D_68=6.0,S_8,R_17,P_4,D_81,D_128,B_37,R_15,D_70,D_71,B_32,S_9,D_93,R_8,S_22,D_61,B_7,D_86,B_26,B_20,S_13,D_126=-1.0,D_126=0.0,D_126=1.0,D_69,R_16,cnt,D_109,D_64=O,D_64=R,D_64=U,D_113,R_23,R_13,B_29,D_133,D_49,B_17,D_39,D_87,D_117=-1.0,D_117=1.0,D_117=2.0,D_117=3.0,D_117=4.0,D_117=5.0,D_117=6.0,D_140,S_3,D_62,D_72,R_19,D_80,D_54,B_38=1.0,B_38=2.0,B_38=3.0,B_38=4.0,B_38=5.0,B_38=6.0,B_38=7.0,D_84,D_116=0.0,D_116=1.0,D_75,D_83,D_111,D_121,D_110,R_26,R_20,D_82,B_13,R_18,D_88,S_26,B_40,R_14,B_21,D_48,R_11,S_17,R_28,R_9,D_92,S_27,D_76,D_96,D_115,D_42,R_5,B_10,D_94,R_27,B_12,D_55,D_107,D_44,S_25,D_51,B_28,D_129,log_B_11,log_B_12,log_B_13,log_B_21,log_B_22,log_B_23,log_B_24,log_B_26,log_B_27,log_B_28,log_B_29,log_B_3,log_B_32,log_B_36,log_B_4,log_B_40,log_B_41,log_B_42,log_B_5,log_B_9,log_D_106,log_D_107,log_D_108,log_D_109,log_D_113,log_D_115,log_D_118,log_D_119,log_D_123,log_D_125,log_D_131,log_D_133,log_D_135,log_D_136,log_D_137,log_D_138,log_D_140,log_D_39,log_D_41,log_D_43,log_D_44,log_D_45,log_D_49,log_D_51
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.005923,0.182713,0.002916,0.006416,0.005271,0.003728,0.0,9.0,0.002312,0.898289,NaN,0.005161,0.005406,0.003851,1.004091,0.008423,0.0,0.0,9.0,0.0,0.0,0.0,1.608184,0.005669,0.446102,0.005773,0.003876,0.009732,1.004421,NaN,0.590665,0.459957,0.136646,0.218059,0.004888,0.002635,0.004177,NaN,NaN,0.005912,0.0,5.0,0.136015,2019-02-19,2019-10-12,235,0.00601,0.005000,0.004620,0.005308,0.047263,0.004567,0.117639,NaN,0.004934,0.005730,0.004126,0.601387,NaN,0.247668,0.591933,0.003792,0.006266,0.020536,NaN,0.145303,0.005657,NaN,0.006835,0.003733,0.011955,0.949735,0.377217,0.170330,0.004632,0.005480,9.0,0.0,0.0,0.005720,NaN,0.004009,0.004483,0.246912,5.0,0.0,0.132557,0.005203,0.005494,0.227071,NaN,1.005949,NaN,NaN,0.104659,0.006187,0.479550,0.412407,0.013925,0.042616,0.006742,0.006821,0.018828,NaN,0.004423,1.006641,NaN,NaN,0.005166,0.005101,0.004591,0.254266,NaN,0.219199,0.004534,0.005501,0.004468,0.0,0.0,0.0,0.0,0.0,5.0,0.271142,0.004437,0.961646,0.004031,0.006024,0.013483,0.003600,0.004415,0.072740,0.005471,0.01557,0.004286,0.006230,0.962658,0.597552,0.283370,0.007949,0.005206,0.005869,0.601419,0.0,9.0,0.0,0.006233,0.004303,9,0.004123,0.0,0.0,5.0,0.005696,0.005964,0.004435,0.003894,0.004254,NaN,NaN,0.069758,NaN,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003996,0.188640,0.035462,0.004783,0.003948,0.005002,1.005194,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.004711,5.0,0.0,0.204130,0.005146,NaN,0.196751,NaN,NaN,0.004758,0.504131,0.054207,0.003736,NaN,0.038056,0.368253,0.004547,0.007769,0.578279,0.005572,0.005725,0.005292,NaN,0.116208,0.299671,NaN,0.004607,0.252104,0.118016,0.004464,0.037069,0.005423,1.006624,0.082867,0.317071,0.002802,0.007083,0.973917,0.041620,0.14522,0.006681,-5.085335,-2.490520,-2.914948,-4.857647,-5.321023,-1.369376,-5.258523,-5.257944,-5.163776,-1.929504,-5.548368,-4.632286,-5.208375,-5.519265,-2.020743,-0.998985,-4.987689,NaN,-3.052031,-5.590489,NaN,-5.877342,NaN,-5.491236,-5.168012,-1.377912,-1.398724,-1.395667,-5.591896,-5.172801,-5.377307,-5.459926,NaN,NaN,NaN,NaN,-5.522559,-2.662719,-5.938794,-4.985657,-4.950068,-4.426577,NaN,-3.179183
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.005631,NaN,0.304190,0.005082,0.005466,0.003205,0.0,13.0,0.235882,1.003704,NaN,0.004744,0.005877,0.00

In [33]:
## fill na
all_test_cols = df.columns.tolist()

for c in cat_feats:
    if c not in all_test_cols:
        df[c] = 0
    else:
        df[c] = df[c].fillna(0)


for _, row in agg_stats.iterrows():
    c = row['feat']
    avg = row['mean']

    if c in float_feats + log_feats:
        df[c] = df[c].fillna(value=avg)
        
for c in log_feats:
    df[c] = df[c].fillna(value=df[c].mean())

In [34]:
float64_cols = df.select_dtypes(include=['float64']).columns.tolist()
df[float64_cols] = np.float32(df[float64_cols].values)
#---convert int64 to int32
int64_cols = df.select_dtypes(include=['int64']).columns.tolist()
df[int64_cols] = np.int32(df[int64_cols].values)

In [35]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 924621 entries, 0 to 924620
Columns: 272 entries, customer_ID to D_68=0.0
dtypes: float32(264), int32(5), object(3)
memory usage: 977.0+ MB


In [37]:
pq.write_table(pa.Table.from_pandas(df), r'amex\agg_test_all_rev.parquet', compression = 'GZIP')